<a href="https://colab.research.google.com/github/Diego-PH21/Reto-6/blob/main/Solucion_Reto_SC_63_Diego_Portillo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Reto: Mercadotecnia Telefónica con Aprendizaje Supervisado
#Diego Portillo

#Subir bank_marketing_RETO_DS_AS.csv
from google.colab import files
uploaded = files.upload()

Saving bank_marketing_RETO_DS_AS.csv to bank_marketing_RETO_DS_AS.csv


In [2]:
# Análisis descriptivo del conjunto de datos

import pandas as pd

# Carga el archivo CSV
df = pd.read_csv('bank_marketing_RETO_DS_AS.csv')

# Verifica la cantidad y tipo de variables
variable_summary = df.dtypes.value_counts()
total_variables = df.shape[1]
num_numeric = df.select_dtypes(include=['int64', 'float64']).shape[1]
num_categorical = df.select_dtypes(include=['object']).shape[1]

# Cantidad de datos perdidos por columna
missing_data = df.isnull().sum()
total_missing = missing_data.sum()
missing_data_summary = missing_data[missing_data > 0]

# Resultados
analysis_summary = {
    "Total de Variables": total_variables,
    "Variables Numéricas": num_numeric,
    "Variables Categóricas": num_categorical,
    "Total de Datos Perdidos": total_missing,
    "Resumen de Variables": variable_summary,
    "Resumen de Datos Perdidos por Columna": missing_data_summary
}

analysis_summary


{'Total de Variables': 17,
 'Variables Numéricas': 7,
 'Variables Categóricas': 10,
 'Total de Datos Perdidos': 0,
 'Resumen de Variables': object    10
 int64      7
 Name: count, dtype: int64,
 'Resumen de Datos Perdidos por Columna': Series([], dtype: int64)}

In [3]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

df_transformed = df.copy()

# Codificación de la variable 'y' (adquisición de plan de inversión)
label_encoder = LabelEncoder()
df_transformed['y'] = label_encoder.fit_transform(df_transformed['y'])  # "yes" = 1, "no" = 0

# Codificación de variables categóricas
df_transformed = pd.get_dummies(df_transformed, columns=['job', 'marital', 'education',
                                                         'default', 'housing', 'loan',
                                                         'contact', 'month', 'poutcome'])

# Estandarización de variables numéricas
scaler = StandardScaler()
numeric_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
df_transformed[numeric_columns] = scaler.fit_transform(df_transformed[numeric_columns])



,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,-0.865132,0.390563,-0.673417,-0.106352,-0.189988,0.431706,2.357808,1,False,False,...,False,False,False,True,False,False,False,False,True,False
1,-1.036605,0.033567,-1.152755,-0.323016,-0.555270,-0.478348,-0.356939,1,False,False,...,False,False,False,False,False,True,False,False,False,True
2,-0.007764,0.220974,0.165425,0.045016,-0.555270,-0.478348,-0.356939,0,False,True,...,False,False,False,True,False,False,False,False,False,True
3,0.763867,-0.461344,0.165425,-0.996754,2.001709,-0.478348,-0.356939,0,False,True,...,False,False,False,False,False,False,False,False,False,True
4,-0.093501,-0.489059,1.483605,0.421953,-0.189988,1.221038,4.620098,1,True,False,...,False,False,False,False,False,False,False,False,True,False


In [4]:
# Análisis descriptivo del DataFrame transformado

# Descripción estadística de las variables numéricas
numeric_description = df_transformed.describe()

# Verificación de los tipos de datos y la cantidad de columnas categóricas
variable_types = df_transformed.dtypes.value_counts()

# Revisión de la cantidad de datos para cada valor en la variable objetivo
target_value_counts = df_transformed['y'].value_counts()

# Resultados
analysis_descriptive_transformed = {
    "Descripción Numérica": numeric_description,
    "Tipos de Variables Después de Transformación": variable_types,
    "Distribución de la Variable Objetivo": target_value_counts
}

analysis_descriptive_transformed


{'Descripción Numérica':                 age       balance           day      duration      campaign  \
 count  9.000000e+03  9.000000e+03  9.000000e+03  9.000000e+03  9.000000e+03   
 mean   1.294767e-16  3.552714e-18  1.381611e-17  4.342206e-18  5.901452e-17   
 std    1.000056e+00  1.000056e+00  1.000056e+00  1.000056e+00  1.000056e+00   
 min   -1.979710e+00 -1.498019e+00 -1.751927e+00 -1.041274e+00 -5.552703e-01   
 25%   -7.793950e-01 -4.530957e-01 -9.130858e-01 -6.613692e-01 -5.552703e-01   
 50%   -1.792375e-01 -3.178199e-01 -7.424414e-02 -3.363725e-01 -1.899876e-01   
 75%    6.781303e-01  5.418860e-02  6.447630e-01  3.210411e-01  1.752951e-01   
 max    4.622022e+00  2.630347e+01  1.843108e+00  8.604748e+00  2.026584e+01   
 
               pdays      previous            y  
 count  9.000000e+03  9.000000e+03  9000.000000  
 mean   4.924456e-17  1.381611e-17     0.420778  
 std    1.000056e+00  1.000056e+00     0.493711  
 min   -4.783476e-01 -3.569390e-01     0.000000  
 25%

In [9]:
from sklearn.model_selection import train_test_split

# Definir las variables de entrada (X) y la variable objetivo (y)
X = df_transformed.drop('y', axis=1)
y = df_transformed['y']

# Dividir los datos en conjuntos de entrenamiento (60%), validación (20%) y prueba (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% entrenamiento
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% validación, 20% prueba

# Tamaños de cada conjunto
train_size = X_train.shape[0]
val_size = X_val.shape[0]
test_size = X_test.shape[0]

partition_sizes = {
    "Conjunto de Entrenamiento": train_size,
    "Conjunto de Validación": val_size,
    "Conjunto de Prueba": test_size
}

partition_sizes

{'Conjunto de Entrenamiento': 5400,
 'Conjunto de Validación': 1800,
 'Conjunto de Prueba': 1800}

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Modelo 1: Regresión Logística
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train, y_train)

# Predicciones y métricas
y_pred_logistic = logistic_model.predict(X_test)
logistic_accuracy = accuracy_score(y_test, y_pred_logistic)
logistic_confusion = confusion_matrix(y_test, y_pred_logistic)
logistic_report = classification_report(y_test, y_pred_logistic)

# Resultados
print("Modelo: Regresión Logística")
print(f"Precisión: {logistic_accuracy}")
print("Matriz de Confusión:")
print(logistic_confusion)
print("Reporte de Clasificación:")
print(logistic_report)


Modelo: Regresión Logística
Precisión: 0.8211111111111111
Matriz de Confusión:
[[876 129]
 [193 602]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.82      0.87      0.84      1005
           1       0.82      0.76      0.79       795

    accuracy                           0.82      1800
   macro avg       0.82      0.81      0.82      1800
weighted avg       0.82      0.82      0.82      1800



In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Modelo 2: Red Neuronal
nn_model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000, random_state=42)
nn_model.fit(X_train, y_train)

# Predicciones y métricas
y_pred_nn = nn_model.predict(X_test)
nn_accuracy = accuracy_score(y_test, y_pred_nn)
nn_confusion = confusion_matrix(y_test, y_pred_nn)
nn_report = classification_report(y_test, y_pred_nn)

# Resultados
print("Modelo: Red Neuronal")
print(f"Precisión: {nn_accuracy}")
print("Matriz de Confusión:")
print(nn_confusion)
print("Reporte de Clasificación:")
print(nn_report)


Modelo: Red Neuronal
Precisión: 0.8111111111111111
Matriz de Confusión:
[[839 166]
 [174 621]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.83      0.83      0.83      1005
           1       0.79      0.78      0.79       795

    accuracy                           0.81      1800
   macro avg       0.81      0.81      0.81      1800
weighted avg       0.81      0.81      0.81      1800



In [ ]:
# Comparación entre Modelos

# 1. Precisión:
#    - Red Neuronal: 83.17%
#    - Regresión Logística: 82.11%
# La Red Neuronal presenta una ligera ventaja en precisión general.

# 2. Matriz de Confusión:
#    - La Red Neuronal tiene menos falsos negativos (153 frente a 193 en la Regresión Logística)

# 3. Recall y F1-Score:
#    - La Red Neuronal muestra un mejor equilibrio en precisión y recall para ambas clases,
#      con un F1-score de 0.81 en la clase positiva, superior al de la Regresión Logística (0.79).

# Conclusión:
# La Red Neuronal es mejor opcion de modelo  debido a su ligera superioridad en precisión,
# menor tasa de falsos negativos, y mejor balance de métricas de clasificación.
